# Machine Learning 2024/2025 - Progress Task 2 (Machine Learning Model Applications and Analysis)

> Final position: **??**

We have tested 2 models for this task, a Neural Network, a Random Forest and a KNN model. More specifically the implementations used were:
* For the neural network:
  * A normal Keras model, with 3 hidden layers.
  * Using a combination of `scikit-learn`'s `RandomSearchCV` model, and a Keras model, to try to find the optimal hyperparameters for the network
  * `MLPClassifier`, a neural network implementation found in `scikit-learn`, instead of being taken from an external library like Keras.
* A Random Forest Model, developed using `scikit-learn`'s implementation. This model was the one with the best performance.
* KNN model <!-- TODO Not read this one, fill this in -->


## Neural Networks

### Keras

### Keras & RandomSearchCV

### MLPClassifier

## Random Forest

## KNN